# Pré-processamento
Este notebook é responsável por pré-processar os dados realizando as seguintes tarefas:
- Definição de tipos
- Tratamento de dados ausentes
- Normalização e discretização
- Limpeza de dados (univariado, bivariado e multivariado)

Este notebook usa o 'clean_vehicles.csv' como dataset, que já não tem mais as colunas 'county', 'region_url' e 'image_url'.

## Definição de tipos
Olhando para a tabela que descreve o dataset em '1-initial-analysis.ipynb' e pela análise feita no notebook, os dados foram definidos nos formatos abaixo.

### Dados categóricos
- type
- region
- transmission
- manufacturer
- model
- condition
- cylinders
- fuel
- title_status
- drive
- size
- paint_color
- state

### Dados do tipo inteiro
- id
- price
- year

### Dados do tipo float
- odometer
- lat
- long

### Dados do tipo object
- url
- description
- vin

In [20]:
def set_df_vehicles_data_types(df):
    data_types = {'category': ['type', 'region', 'transmission', 'manufacturer', 'model', 'condition', 'cylinders', 'fuel', 'title_status', 'drive', 'size', 'paint_color', 'state'],
              'int': ['id', 'price', 'year'],
              'float': ['odometer', 'lat', 'long'],
              'object': ['url', 'description', 'vin']}
    
    for key in data_types.keys():
        for elem in data_types[key]:
            df[elem] = df[elem].astype(key)

In [8]:
from Utils import *

DATASET = "../datasets/clean_vehicles.csv"
df = pd.read_csv(DATASET)

In [6]:
price_series = df['price'].apply(get_decimal)
price_series = price_series[price_series > 0]

In [7]:
price_series

Series([], Name: price, dtype: int64)

In [16]:
df['price'].astype('object')

0          3495
1         13750
2          2300
3          9000
4             0
          ...  
435844     2500
435845     2700
435846     2450
435847     8995
435848     6577
Name: price, Length: 435849, dtype: object

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435849 entries, 0 to 435848
Data columns (total 25 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   id            435849 non-null  int64   
 1   url           435849 non-null  object  
 2   region        435849 non-null  category
 3   region_url    435849 non-null  object  
 4   price         435849 non-null  int64   
 5   year          434732 non-null  float64 
 6   manufacturer  415102 non-null  category
 7   model         429650 non-null  category
 8   condition     249043 non-null  category
 9   cylinders     269465 non-null  category
 10  fuel          432858 non-null  category
 11  odometer      360701 non-null  float64 
 12  title_status  434043 non-null  category
 13  transmission  433703 non-null  category
 14  vin           239197 non-null  object  
 15  drive         313838 non-null  category
 16  size          139888 non-null  category
 17  type          318741 non-null

In [21]:
set_df_vehicles_data_types(df)
df.dtypes

ValueError: Cannot convert non-finite values (NA or inf) to integer